In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
embedding = {}
f = open('/kaggle/input/glove6b100dtxt/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    em = np.asarray(values[1:], dtype='float32')
    embedding[word] = em
f.close()

In [ ]:
data_train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
data_test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")

In [ ]:
data_train.head()

In [ ]:
# Apostrophe Dictionary(Từ điển viết tắt)
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

# Function replace acronyms (Hàm xử lý từ viết tắt -> từ đầy đủ)
def lookup_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [ ]:
data_train['excerpt'] = data_train['excerpt'].apply(lambda x: lookup_dict(x,apostrophe_dict))
data_test['excerpt'] = data_test['excerpt'].apply(lambda x: lookup_dict(x,apostrophe_dict))

In [ ]:
from sklearn.model_selection import train_test_split

y = data_train['target']
data_train = data_train['excerpt']

data_train, data_val, y, y_val = train_test_split(
    data_train.values.tolist(), y.values.tolist(), test_size = 0.20, random_state = 12)

In [ ]:
def average_doc_vec(words):
  a = [embedding[w] for w in words if w in embedding]
  a = np.asarray(a)
  return np.sum(a, 0)/len(a)

In [ ]:
data_train[5].split()

In [ ]:
import numpy as np
n = len(data_train)
m = 100
n_val = len(data_val)
n_test = len(data_test)
X = np.zeros((n, m))
X_val = np.zeros((n_val, m))
X_test = np.zeros((n_test, m))

import string

for i in range(n):
    tokens = data_train[i].split()
    X[i,:] = average_doc_vec(tokens)

for i in range(n_val):
    tokens_val = data_val[i].split()
    X_val[i, :] = average_doc_vec(tokens_val)
    
for i in range(n_test):
    tokens_test = data_test['excerpt'][i].split()
    X_test[i, :] = average_doc_vec(tokens_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lr = LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X_val)

mean_squared_error(y_pred, y_val)

In [ ]:
from sklearn.linear_model import Ridge

rr = Ridge(alpha=0.759, max_iter=500)
rr.fit(X, y)

y_pred = rr.predict(X_val)

mean_squared_error(y_pred, y_val)

In [ ]:
y_pred = lr.predict(X_test)
my_submission = pd.DataFrame({'id': data_test.id.to_numpy(), 'target': y_pred })
my_submission.to_csv('submission.csv', index=False)